# Which csv?

## database.csv or fatalencounters??? or both??

We are going to try to see if we can use the name as a temporary key to mix the two databases

In [18]:
# Importing pandas

import pandas as pd
import datetime as dt

In [19]:
# Importing the 2 databases

wpost = pd.read_csv ('/Users/ReDi/Documents/GitHub/data-analysis-project/data/database.csv')
fenc = pd.read_csv ('/Users/ReDi/Documents/GitHub/data-analysis-project/data/fatalencounters.csv')

In [20]:
# Let´s have a quick look at the 2 databases.
# The Washington Post one:

wpost.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2142 entries, 0 to 2141
Data columns (total 14 columns):
id                         2142 non-null int64
name                       2123 non-null object
date                       2142 non-null object
manner_of_death            2142 non-null object
armed                      2136 non-null object
age                        2099 non-null float64
gender                     2141 non-null object
race                       2039 non-null object
city                       2142 non-null object
state                      2142 non-null object
signs_of_mental_illness    2142 non-null bool
threat_level               2142 non-null object
flee                       2106 non-null object
body_camera                2142 non-null bool
dtypes: bool(2), float64(1), int64(1), object(10)
memory usage: 205.1+ KB


In [21]:
# And now the Fatal encounters one:

fenc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20792 entries, 0 to 20791
Data columns (total 24 columns):
Timestamp                                                                          20791 non-null float64
Subject's name                                                                     20792 non-null object
Subject's age                                                                      20207 non-null object
Subject's gender                                                                   20757 non-null object
Subject's race                                                                     20791 non-null object
URL of image of deceased                                                           7084 non-null object
Date of injury resulting in death (month/day/year)                                 20792 non-null object
Location of injury (address)                                                       20315 non-null object
Location of death (city)                           

At first glance it can appear the second one to be more in everything, more entries, more columns. The first thing is true: we have indeed more entries...but...more columns?

Name/ age / gender / race, are common to both databases ... so 4 - 4

talking about place: Wpost has city / state
                     Fenc has address/city/state/zip/county
                                                        ... so 6 - 9  (3 columns of additional info about location in fenc)
                                                        
We have date, manner (or cause) of death and mental illness in both..9 - 12

We have 4 unique columns in wpost:
Armed / Threat level / Flee / Body Camera                   ...13 - 12  ( 4 columns exclusive of wpost)

We have several unique columns in fenc, but some expendable:
URL of image of deceased / A brief description of the circumstances surrounding the death / Link to news article or photo of official document / State Data Status / Date&Description / Date (Year) / 20791 are expendable (we can keep the description for curiosity purposes)
We have 2 identifiers: de unique id and the timestamp ( we just need one)
We have 2 uniqe columns: agency and official dispotition    ...so 13 - 14  (2 uniques in fenc)


We have 5 columns unique in the fenc (3 of them about the location), and 4 unique in the wpost

In [22]:
# Comparing the 2 databases entries (just a remembrance)

wpost.shape, fenc.shape

((2142, 14), (20792, 24))

### Time frame of each database

Let´s see, more or less, wich is the difference of entries between our 2 databases considering a similar time frame.

In [23]:
fenc['Date (Year)'].min(), fenc['Date (Year)'].max()

(2000, 2100)

In [24]:
# Since we have a problem here, let´s see what is going to happen in the 2100


fenc[fenc['Date (Year)'] > 2017]

,Timestamp,Subject's name,Subject's age,Subject's gender,Subject's race,URL of image of deceased,Date of injury resulting in death (month/day/year),Location of injury (address),Location of death (city),Location of death (state),...,A brief description of the circumstances surrounding the death,Official disposition of death (justified or other),Link to news article or photo of official document,Symptoms of mental illness?,Unique identifier/submitted by,"State Data Status Prior to Jan. 1, 2013. All states complete after Jan. 1, 2013",Date&Description,20791,Unique identifier,Date (Year)
20791,NaN,Items below this row have not been fact-checked.,NaN,NaN,NaN,NaN,"December 31, 2100",Items below this line have not been fact-checked.,NaN,NaN,...,NaN,NaN,http://www.fatalencounters.org,NaN,Bill jones,X,12/31/2100: http://www.fatalencounters.org,20791.0,NaN,2100


In [25]:
# So, we have like and ending entrie that has no value to our database, le´s get rid of it

fenc = fenc.drop(20791)

In [26]:
fenc['Date (Year)'].min(), fenc['Date (Year)'].max()

(2000, 2017)

So our fatal encounters database goes from 2000 to 2017. We have solved the error of the 2100 entry dropping it.

In [27]:
# Let´s see the time frame of wpost, and the number of entries

wpost.date.min(), wpost.date.max(), wpost.id.count()

('2015-01-02', '2017-02-28', 2142)

In [28]:
# Approx, in the same time frame, in the fenc:

fenc[fenc['Date (Year)'] >= 2015].shape

(3919, 24)

We can see here that approximately, the fatal encounters db is doble the size of the washington post one. Let's try to be more accurate.

Since our column about the date of the killing in the fenc database is a string, and not a date, we cannot use it. We have to transform the data. In order to do that e are going to use the datetime library. We are going to use it to give format of a date to this column. We are going to create a new column where we are going to have this date data in a proper way.



In [30]:
# We iterate through every row, using iterrows. First we create our new column, with a string type

for i, row in fenc.iterrows():

    # we define n, as our string to be transformed for every entry
    n = row['Date of injury resulting in death (month/day/year)']

    # and here we define our new column, date, that is going to be n transformed into date. For doing that, we have to
    # describe which is the original format. In our case the date is written on a string like 'January 23, 2005'.
    # If we go to the documentation of the datetime library, we see that %B stands for the complete name of the month,
    # %d stands for the day number, then a coma, and the the year in 4 digits format.
    # The date in the end is to have just the date, without hours minutes and seconds
    fenc['date'].iloc[i] = dt.datetime.strptime(n,'%B %d, %Y')

C:\Users\ReDi\Anaconda3\lib\site-packages\pandas\core\indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [33]:
fenc[(fenc['date'] >= '2015-01-02') & (fenc['date'] <= '2017-02-28')].shape

(3420, 25)

So in the end, for the same time frame the washington post dataframe is 2142 entries, the fatal encounters one is 3420. Can we try to merge them? Since the second one is 1300 entries bigger, we can asume that every name in the wpost is indeed in the fatal encounters one...but we can have typos, names that don´t match, missing names...let´s investigate.


## Merging the csv's

Just for future uses, and because my computer is very slow, let's save this as a new csv

In [38]:
fenc.to_csv('fenc.csv', index = False, encoding='utf-8')

,Timestamp,Subject's name,Subject's age,Subject's gender,Subject's race,URL of image of deceased,Date of injury resulting in death (month/day/year),Location of injury (address),Location of death (city),Location of death (state),...,Official disposition of death (justified or other),Link to news article or photo of official document,Symptoms of mental illness?,Unique identifier/submitted by,"State Data Status Prior to Jan. 1, 2013. All states complete after Jan. 1, 2013",Date&Description,20791,Unique identifier,Date (Year),date
0,1.0,LaTanya Janelle McCoy,24,Female,Race unspecified,NaN,"January 2, 2000",5700 block Mack Road,Sacramento,CA,...,Unknown,http://www.recordnet.com/article/20000110/A_NE...,No,NaN,Complete,1/2/2000: LaTanya Janelle McCoy's car was stru...,NaN,1.0,2000,2000-01-02
1,2.0,Lester Miller,53,Male,Race unspecified,NaN,"January 2, 2000",4850 Flakes Mill Road,Ellenwood,GA,...,Criminal,https://docs.google.com/document/d/1-YuShSarW_...,No,NaN,Complete,"1/2/2000: Darren Mayfield, a DeKalb County she...",NaN,2.0,2000,2000-01-02
2,19512.0,James Martin Loy,55,Male,Race unspecified,NaN,"January 3, 2000",107 South Grant St.,Uniontown,PA,...,Suicide,https://docs.google.com/document/d/1hCD2mqtJeq...,No,NaN,Complete,1/3/2000: Loy and another motorist were appare...,NaN,19512.0,2000,2000-01-03
3,4.0,John Edward Pittman,45,Male,African-American/Black,NaN,"January 5, 2000",NaN,Dothan,AL,...,Justified,https://news.google.com/newspapers?nid=1817&da...,Unknown,0@CRDeCeros/,Complete,1/5/2000: John Edward Pittman was shot and kil...,NaN,4.0,2000,2000-01-05
4,5.0,John Frank Brown,20,Male,African-American/Black,http://media.cmgdigital.com/shared/img/photos/...,"January 5, 2000",56 Jesse Hill Jr Dr,Atlanta,GA,...,Accidental,http://www.wsbtv.com/news/news/fatal-cop-beati...,Drug or alcohol use,johndoe20/ Lauren,Complete,"1/5/2000: On Jan. 5 around midnight, 20-year-o...",NaN,5.0,2000,2000-01-05
5,3.0,Derrick E. Tate,23,Male,Race unspecified,NaN,"January 5, 2000",1900 block W Reynolds St,Pontiac,IL,...,Justified,https://drive.google.com/file/d/0B62PuBu1vKRLY...,No,CRDeCeros,Complete,1/5/2000: Officer Elias E. Mendiola shot Derri...,NaN,3.0,2000,2000-01-05
6,8.0,Adrian Dolby,31,Male,Race unspecified,NaN,"January 6, 2000",3400 block Gray Street,Muscoy,CA,...,Unknown,https://docs.google.com/document/d/1DgZgZi6c4t...,Unknown,NaN,Complete,1/6/2000: SWAT officers shot and killed Adrian...,NaN,8.0,2000,2000-01-06
7,7.0,Darryl Woodall Jr.,23,Male,Race unspecified,NaN,"January 6, 2000",Mulberry Church Rd & I-85,Charlotte,NC,...,Unreported,https://groups.google.com/forum/#!topic/alt.tr...,No,CRDeCeros/Walt Lockley,Complete,1/6/2000: State troopers and county deputies h...,NaN,7.0,2000,2000-01-06
8,6.0,Kyle Dillon,19,Male,European-American/White,NaN,"January 6, 2000",1700 block North York Avenue,York,NE,...,Grand jury/No bill or Cleared,http://www.theindependent.com/news/million-dol...,No,0@CRDeCeros/,Complete,1/6/2000: Kyle Dillon allegedly walked up to o...,NaN,6.0,2000,2000-01-06
9,10.0,Joseph Gumpert,28,Male,Race unspecified,NaN,"January 7, 2000",9600 block Cheddar Street,Downey,CA,...,Unknown,http://articles.latimes.com/2000/jan/10/local/...,Unknown,NaN,Complete,1/7/2000: Joseph Gumpert stopped breathing aft...,NaN,10.0,2000,2000-01-07
